<a href="https://colab.research.google.com/github/akam7044/thesis-2023-athena/blob/bert_embeddings/BERT_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating BERT embeddings 


### Run this section when using google collab


In [44]:
# !pip install cloud-tpu-client==0.10 torch==2.0.0 torchvision==0.15.1 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp310-cp310-linux_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.9/162.9 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━

In [45]:
# # imports pytorch
# import torch

# # imports the torch_xla package
# import torch_xla
# import torch_xla.core.xla_model as xm

In [50]:
# # Creates a random tensor on xla:1 (a Cloud TPU core)
# device = xm.xla_device()
# t1 =torch.arange(0, 100, device=device)
# print(t1)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
        90, 91, 92, 93, 94, 95, 96, 97, 98, 99], device='xla:1')


## Investigate the token problem 
See what the max tokens is for each csv

In [82]:
%pip install transformers


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [83]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [85]:
import pandas as pd 

df_whisper = pd.read_csv('spontaneousDialougeOnly_whisper.csv')
df_wav2vec = pd.read_csv('spontaneousDialougeOnly_wav2vec.csv')
df_google  = pd.read_csv('spontaneousDialogueOnly_google.csv')

display(df_whisper['transcripts'].head())
display(df_wav2vec['transcripts'].head())
display(df_google['transcripts'].head())

0     Yeah, in London you can go to Oxford Street, ...
1     Okay. Okay. Yes. Okay. So you want to know wh...
2     So this is your first time in London, you've ...
3     Okay, so I'm much very long than you're here....
4     Okay, so I'm much very long than you're here....
Name: transcripts, dtype: object

0    Ye m in london ygoi to  oxford streets which i...
1    O k k yes o k am am so you want you want to kn...
2    So this is your your first time in london you'...
3    E i so i am not very  londoner here i camp her...
4    I iso i am not very londoner here i came here ...
Name: transcripts, dtype: object

0    Yeah, I'm in London. You can go to Oxford Stre...
1    Okay.  Okay. Yes. Okay, so you want you want t...
2    So this is your your first time in London, you...
3    Okay. So I'm a Londoner fewer. I came here for...
4    Okay. So I'm a Londoner fewer. I came here for...
Name: transcripts, dtype: object

In [86]:
import nltk.data
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

def find_max_token (df):
    max_len = 0
    transcripts = df['transcripts'].values

    for transcript in transcripts:
        #transcript = "[SEP]".join(sent_detector.tokenize(transcript.strip())) #for each sentence we need a sentence seperator operator 
        input_ids = tokenizer.encode(transcript,add_special_tokens=True)
        max_len = max(max_len,len(input_ids))
    
    return max_len

max_len_whisper = find_max_token(df_whisper)
max_len_wav2vec = find_max_token(df_wav2vec)
max_len_google = find_max_token(df_google)

display('Max token whisper:',max_len_whisper)
display('Max token wav2vec:',max_len_wav2vec)
display('Max token google:',max_len_google)

    

'Max token whisper:'

390

'Max token wav2vec:'

420

'Max token google:'

378

All fit within the max limit for 516 so we are okay. The max token will be set to 450 just incase there is any problem. 


In [87]:
print(len(tokenizer.tokenize(df_whisper.transcripts.values[10])))
print(len(tokenizer.encode_plus(df_whisper.transcripts.values[10])['input_ids']))

184
186


So the encoder doesn't split the transcript into sentences ...

So to start off with I will generate embeddings for the WHOLE transcript

### Generate Tokens for the whole transcript - testing with one first

http://mccormickml.com/2019/07/22/BERT-fine-tuning/

In [ ]:
# import torch
# from torch.utils.data import TensorDataset, random_split
# from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [94]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


### Whisper 


In [95]:
df_whisper = pd.read_csv('spontaneousDialougeOnly_whisper.csv')

transcripts = df_whisper.transcripts.values
labels = df_whisper.classification.values

In [109]:
len(df_whisper)

57

In [110]:
for i in range(5):
    print(i)

0
1
2
3
4


In [98]:
import nltk.data
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

input_ids = []
attention_masks = []
count = []

for transcript in transcripts:

    #transcript = "[SEP]".join(sent_detector.tokenize(transcript.strip()))
    count.append(len(sent_detector.tokenize(transcript.strip())))

    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        transcript,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 450,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', transcripts[0])
print('Token IDs:', input_ids[0])
print('Attention:', attention_masks[0])
print(count)


Original:   Yeah, in London you can go to Oxford Street, which is famous for shopping and there's self-aguse there and a lot of tourists come there and a lot there. So it's a good place to see when you come to London and there's Buckingham Palace where the Queen lives and that's a good place. That's the Royal Family lives so you can come there. And there's other, there's Big Ben, the houses of Parliament where the government, the government are, British government are. So that's a good place to go and there's the London Eye where you can see the whole of London, you go and you can see the whole of London. So that's a good place to visit in London. Yeah, so if I was, I would probably, I would go by public transport then you would see how London really is and you would see the whole of London if you go in a bus or the transport you would see how London is. So I would advice you go on public transport buses and trains so you would really enjoy it. You would get on a like an open tour bus 

/var/folders/zx/41yyp8n50255b1fh_dg2stwr0000gn/T/ipykernel_50190/3711164511.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels)


In [99]:
#print(input_ids[0])
input_id_resize = input_ids[0].reshape(1,450)
attention_masks_resize = attention_masks[0].reshape(1,450)
print(input_id_resize)

tensor([[  101,  3398,  1010,  1999,  2414,  2017,  2064,  2175,  2000,  4345,
          2395,  1010,  2029,  2003,  3297,  2005,  6023,  1998,  2045,  1005,
          1055,  2969,  1011, 12943,  8557,  2045,  1998,  1037,  2843,  1997,
          9045,  2272,  2045,  1998,  1037,  2843,  2045,  1012,  2061,  2009,
          1005,  1055,  1037,  2204,  2173,  2000,  2156,  2043,  2017,  2272,
          2000,  2414,  1998,  2045,  1005,  1055, 17836,  4186,  2073,  1996,
          3035,  3268,  1998,  2008,  1005,  1055,  1037,  2204,  2173,  1012,
          2008,  1005,  1055,  1996,  2548,  2155,  3268,  2061,  2017,  2064,
          2272,  2045,  1012,  1998,  2045,  1005,  1055,  2060,  1010,  2045,
          1005,  1055,  2502,  3841,  1010,  1996,  3506,  1997,  3323,  2073,
          1996,  2231,  1010,  1996,  2231,  2024,  1010,  2329,  2231,  2024,
          1012,  2061,  2008,  1005,  1055,  1037,  2204,  2173,  2000,  2175,
          1998,  2045,  1005,  1055,  1996,  2414,  

In [100]:
# Get all of the model's parameters as a list of tuples.
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

params = list(model.named_parameters())

print(len(params))

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


199
The BERT model has 199 different named parameters.

==== Embedding Layer ====

embeddings.word_embeddings.weight                       (30522, 768)
embeddings.position_embeddings.weight                     (512, 768)
embeddings.token_type_embeddings.weight                     (2, 768)
embeddings.LayerNorm.weight                                   (768,)
embeddings.LayerNorm.bias                                     (768,)

==== First Transformer ====

encoder.layer.0.attention.self.query.weight               (768, 768)
encoder.layer.0.attention.self.query.bias                     (768,)
encoder.layer.0.attention.self.key.weight                 (768, 768)
encoder.layer.0.attention.self.key.bias                       (768,)
encoder.layer.0.attention.self.value.weight               (768, 768)
encoder.layer.0.attention.self.value.bias                     (768,)
encoder.layer.0.attention.output.dense.weight             (768, 768)
encoder.layer.0.attention.output.dense.bias                

In [101]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
print(input_id_resize.size())
with torch.no_grad():

    outputs = model(input_id_resize, attention_masks_resize)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

torch.Size([1, 450])


In [102]:
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 450
Number of hidden units: 768


In [103]:
# `hidden_states` is a Python list.
print('      Type of hidden_states: ', type(hidden_states))

# Each layer in the list is a torch tensor.
print('Tensor shape for each layer: ', hidden_states[0].size())

      Type of hidden_states:  <class 'tuple'>
Tensor shape for each layer:  torch.Size([1, 450, 768])


In [104]:
# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(hidden_states, dim=0)

token_embeddings.size()

torch.Size([13, 1, 450, 768])

In [75]:
# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()

torch.Size([13, 450, 768])

In [76]:
# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

torch.Size([450, 13, 768])

In [80]:
# `hidden_states` has shape [13 x 1 x 22 x 768]

# `token_vecs` is a tensor with shape [22 x 768]
token_vecs = hidden_states[-2][0]

# Calculate the average of all 22 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)

print(sentence_embedding.size())

torch.Size([768])


In [111]:
print(sentence_embedding)

tensor([ 9.2898e-02, -3.1706e-01,  3.0769e-01,  5.9216e-02,  1.2851e-01,
        -1.6297e-01, -6.7700e-02,  1.1933e+00, -1.5987e-01, -3.2072e-01,
        -2.2333e-01, -3.9926e-02,  6.7172e-02,  1.7070e-01, -6.8330e-02,
        -4.3218e-02,  2.2641e-01,  3.3407e-01,  1.4887e-01,  4.6192e-01,
         4.6184e-01,  4.2969e-01, -4.1225e-01, -2.2601e-01,  3.5624e-01,
         2.4428e-01,  1.6205e-01, -2.3265e-01, -1.1740e-01,  2.9191e-01,
         3.9149e-01,  1.8550e-01,  1.8248e-02, -2.1975e-01,  1.2000e-01,
        -4.7026e-01,  9.5347e-02,  9.6847e-02, -4.5526e-01, -2.2310e-01,
        -2.0141e-01, -2.6811e-01,  3.7789e-01, -3.0233e-01, -3.9713e-01,
         1.5721e-02,  4.5926e-01, -2.8779e-01, -1.1606e-02, -4.4356e-01,
        -2.6350e-01, -1.9754e-01, -4.6238e-01,  4.3067e-02,  4.6804e-01,
        -1.5323e-04, -1.2497e-01, -1.5853e-01, -4.8164e-01,  3.5797e-02,
        -8.8510e-02, -2.4844e-01,  2.9803e-02, -1.2080e-01,  3.1578e-01,
        -2.0020e-01,  4.0094e-01,  5.1318e-02, -4.2

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))
